Combi of
* https://anaconda.org/jbednar/nyc_taxi-paramnb/notebook
* http://datashader.org/topics/param_dashboard.html
* https://github.com/ioam/jupytercon2017-holoviews-tutorial/blob/master/notebooks/01-workflow-introduction.ipynb

Download ny taxi dataset
```
wget http://s3.amazonaws.com/datashader-data/nyc_taxi.zip
unzip nyc_taxi.zip
```    

In [ ]:
import holoviews as hv, geoviews as gv, param, parambokeh, paramnb, dask.dataframe as dd

from colorcet import cm
from bokeh.models import WMTSTileSource
from holoviews.operation.datashader import datashade
from holoviews.streams import RangeXY

hv.extension('bokeh')

In [ ]:
from dask.distributed import Client, LocalCluster

In [ ]:
client = Client(LocalCluster(ip='0.0.0.0', processes=False))
client

In [ ]:
%time df = dd.read_csv('nyc_taxi.csv',usecols= \
                       ['pickup_x', 'pickup_y', 'dropoff_x','dropoff_y', 'passenger_count','tpep_pickup_datetime', 'tpep_dropoff_datetime'])
df['pickup_hour'] = dd.to_datetime(df['tpep_pickup_datetime']).dt.hour
df['dropoff_hour'] = dd.to_datetime(df['tpep_dropoff_datetime']).dt.hour
df = df.persist()

In [ ]:
url='https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg'
tiles = gv.WMTS(WMTSTileSource(url=url))
options = dict(width=900,height=600,xaxis=None,yaxis=None,bgcolor='black',show_grid=False)
map_tiles = tiles.opts(style=dict(alpha=0.75), plot=options)
max_pass = int(df.passenger_count.max().compute()+1)

In [ ]:
class NYCTaxiExplorer(hv.streams.Stream):
    colormap   = param.ObjectSelector(default=cm["fire"], objects=[cm[k] for k in cm.keys() if not '_' in k])
    hour       = param.Integer(default=None, bounds=(0, 23), doc="All hours by default; drag to select one hour")
    passengers = param.Range(default=(0,max_pass), bounds=(0,max_pass))
    location   = param.ObjectSelector(default='dropoff', objects=['dropoff', 'pickup'])

    def make_view(self, x_range=None, y_range=None, **kwargs):
        points = hv.Points(df, kdims=[self.location+'_x', self.location+'_y'], vdims=[self.location+'_hour'])
        selection = {self.location+"_hour":self.hour if self.hour else (0,24), "passenger_count":self.passengers}
        taxi_trips = datashade(points.select(**selection), x_sampling=1, y_sampling=1, cmap=self.colormap,
                               dynamic=False, x_range=x_range, y_range=y_range, width=1000, height=600)
        return taxi_trips.opts(plot=options)

In [ ]:
explorer = NYCTaxiExplorer(name="NYC Taxi Trips")
paramnb.Widgets(explorer, callback=explorer.event)
dmap = hv.DynamicMap(callback=explorer.make_view, streams=[explorer, RangeXY()])

In [ ]:
map_tiles * dmap